In [23]:
import torch

from transformers import EsmModel, AutoTokenizer
from peft import LoraConfig, get_peft_model

In [ ]:
# "facebook/esm2_t33_650M_UR50D"
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
model = EsmModel.from_pretrained("facebook/esm2_t6_8M_UR50D")

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.save_pretrained("../data/esm2-8M",) 
tokenizer.save_pretrained("../data/esm2-8M",)

('../data/esm2-8M/tokenizer_config.json',
 '../data/esm2-8M/special_tokens_map.json',
 '../data/esm2-8M/vocab.txt',
 '../data/esm2-8M/added_tokens.json')

In [14]:
seq = ["SEQVENCE", "ANOTHERSEQ"]

In [19]:
inputs = tokenizer(seq, return_tensors="pt", truncation=True, padding='max_length', max_length=100)
with torch.no_grad():
    output = model(**inputs)

In [20]:
output.last_hidden_state.shape

torch.Size([2, 100, 320])

In [21]:
inputs

{'input_ids': tensor([[ 0,  8,  9, 16,  7,  9, 17, 23,  9,  2,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 0,  5, 17, 28, 11, 21,  9, 10,  8,  9, 16,  2,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,

In [26]:
peft_config = LoraConfig(
            task_type="FEATURE_EXTRACTION",
            inference_mode=False,
            r=4,
            lora_alpha=1,
            bias="all",
            target_modules=["query", "key", "value", "fc1", "fc2"],
            #lora_dropout=0.1,
        )
model.train()
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 67,841 || all params: 7,557,881 || trainable%: 0.8976
